In [ ]:
!pip install selenium

In [5]:
from selenium import webdriver
import pandas as pd
driver = webdriver.Chrome('C:/Windows/chromedriver_win32/chromedriver.exe')

In [6]:
df = pd.DataFrame(columns=['username', 'date', 'comments'])
for j in range(250):
  driver.get('https://forums.edmunds.com/discussion/2864/general/x/entry-level-luxury-performance-sedans/p'+str(j+1))
  for i in range(20):
    df.loc[i+j*20, 'username'] = driver.find_elements_by_xpath('/html/body/div[1]/main/div/div[3]/div[5]/div[1]/ul/li['+ str(i+1) +']/div/div[2]/div[1]/span[1]/a[2]')[0].text
    df.loc[i+j*20, 'date'] = driver.find_elements_by_xpath('/html/body/div[1]/main/div/div[3]/div[5]/div[1]/ul/li['+ str(i+1) +']/div/div[2]/div[2]/span/a/time')[0].text
    df.loc[i+j*20, 'comments'] = driver.find_elements_by_xpath('/html/body/div[1]/main/div/div[3]/div[5]/div[1]/ul/li['+ str(i+1) +']/div/div[3]/div/div[1]')[0].text

In [8]:
#Convert to csv file
import copy
comments_clean = copy.deepcopy(df)
comments_clean['comments'] = comments_clean['comments'].apply(lambda x :x.replace("\n"," "))
comments_clean.to_csv('comments.csv', header=True, sep=',')